## Four in line DeepLearning AI

#### Install dependencies

In [ ]:
!pip install numpy
!pip install tensorflow==2.2
!pip install keras==2.3.1

#### Start by importing libraries

In [ ]:
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model
from keras.layers import Dense, ZeroPadding2D, Activation, Dropout, Input, BatchNormalization, Reshape, Flatten, Conv2D
from keras.optimizers import Adam

#### This is the whole data set

In [ ]:
fileInput = np.genfromtxt('./data-generation/1.txt', delimiter=',')
fileInput.shape

#### We'll slice it into our input, output and train, test, dev sets

In [ ]:
X_train = np.reshape(fileInput[:230000, :42], (230000, 6, 7, 1))
Y_train = np.reshape(fileInput[:230000, 42:], (230000, 1))
X_test = np.reshape(fileInput[230000:235000, :42], (5000, 6, 7, 1))
Y_test = np.reshape(fileInput[230000:235000, 42:], (5000, 1))
X_dev = np.reshape(fileInput[235000:235762, :42], (762, 6, 7, 1))
Y_dev = np.reshape(fileInput[235000:235762, 42:], (762, 1))

In [ ]:
print('input train set:', X_train.shape)
print('output train set:', Y_train.shape)
print('input test set:', X_test.shape)
print('output test set:', Y_test.shape)
print('input dev set:', X_dev.shape)
print('output dev set:', Y_dev.shape)

#### Build model

In [ ]:
def model(input_shape):
    """
    Function creating the model's graph in Keras.
    
    Argument:
    input_shape -- shape of the model's input data (using Keras conventions)

    Returns:
    model -- Keras model instance
    """
    
    X_input = Input(shape = input_shape)
    
    ### START CODE HERE ###
    
    # Step 1: Dense layer (≈4 lines)
    X = ZeroPadding2D(4)(X_input)
    X = Conv2D(8, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)
    X = Conv2D(16, (5, 5), strides = (1, 1), name = 'conv1')(X)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)
    X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)
    X = Flatten()(X)
    X = Dense(32, activation='relu', name = 'dense0')(X)
    X = Dropout(.8)(X)
    X = Dense(1, activation='sigmoid', name = 'dense1')(X)

    ### END CODE HERE ###

    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [ ]:
filModel = model(X_train.shape[1:])

In [ ]:
filModel.compile(optimizer='adam', loss='binary_crossentropy', metrics= ['accuracy'])

#### Optional: Load the model
If loaded, training is not nessesary, but it can be done to try to achieve higher performance

In [ ]:
filModel = load_model('./weights.h5')

#### Train Model
1 epoch -> ~0.59 accuracy on test set
5 epoch -> ~0.60 accuracy on test set


In [ ]:
filModel.fit(x=X_train, y=Y_train, epochs=5, batch_size=64)

### Optional: save the model

In [ ]:
filModel.save('./weights.h5')

#### Test after training

In [ ]:
### START CODE HERE ### (1 line)
preds = filModel.evaluate(x = X_test, y = Y_test)
### END CODE HERE ###
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
x = np.reshape(X_dev[1,:], (1, 6, 7, 1))
print(x.shape)
print('prediction:', filModel.predict(x))
print('correct:', Y_dev[1])